# 라이브러리 임포트

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from keras.layers import Dense, Concatenate,Dropout, Input, Embedding, LSTM, Bidirectional,Flatten,SpatialDropout1D, CuDNNLSTM ,Conv1D,GlobalMaxPooling1D
from keras.models import Model
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,SpatialDropout1D
from sklearn.preprocessing import LabelBinarizer
import keras
import keras.preprocessing.text
from keras.preprocessing import sequence
from random import *
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,SpatialDropout1D, MaxPooling1D
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import copy
from sklearn.metrics import classification_report
from keras.models import load_model
import keras.backend.tensorflow_backend as K
import tensorflow as tf
import pickle

Using TensorFlow backend.


### 모델 로드

In [2]:
n = 150000
model = load_model('./c_model/c_senti_model1.h5')

pos_tagger1 = Mecab()
pos_tagger2 = Twitter()

pos_tagger = pos_tagger1

max_slen1 = 200
max_slen2 = 500
max_slen3 = 200
max_slen4 = 200

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [3]:
# 예측 

with open('./c_model/tk1_1.pickle', 'rb') as handle:
    tk1 = pickle.load(handle)
        
with open('./c_model/tk2_1.pickle', 'rb') as handle:
    tk2 = pickle.load(handle)
    
with open('./c_model/tk3_1.pickle', 'rb') as handle:
    tk3 = pickle.load(handle)
    
with open('./c_model/tk4_1.pickle', 'rb') as handle:
    tk4 = pickle.load(handle)    

In [4]:
BASE_CODE, CHOSUNG, JUNGSUNG = 44032, 588, 28

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


def convert(test_keyword):
    split_keyword_list = list(test_keyword)
    #print(split_keyword_list)

    result = list()
    for keyword in split_keyword_list:
        # 한글 여부 check 후 분리
       # print(keyword)
        if re.match('.*[ㄱ-ㅎㅏ-ㅣ가-힣]+.*', keyword) is not None:
            try:
                char_code = ord(keyword) - BASE_CODE
                char1 = int(char_code / CHOSUNG)
                if char1==0:  
                    result.append('')
                result.append(CHOSUNG_LIST[char1])
               # print('초성 : {}'.format(CHOSUNG_LIST[char1]))
                char2 = int((char_code - (CHOSUNG * char1)) / JUNGSUNG)
                if char2==0:
                    result.append('')
                result.append(JUNGSUNG_LIST[char2])
              #  print('중성 : {}'.format(JUNGSUNG_LIST[char2]))
                char3 = int((char_code - (CHOSUNG * char1) - (JUNGSUNG * char2)))
                if char3==0:
                    result.append('')
                else:
                    result.append(JONGSUNG_LIST[char3])
               # print('종성 : {}'.format(JONGSUNG_LIST[char3]))
            except:
                result.append(keyword)
        else:
            result.append(keyword)
    # result
    return "".join(result)

# 부정 댓글 확률 계산

In [12]:
test = pd.read_csv("./neg_df.csv")
X_test = test['내용']
X_test = X_test.drop_duplicates()
X_test = X_test.reset_index(drop=True)
idx_list = []

# 형태소 분석이 어려운 샘플은 제외
for idx,t in enumerate(X_test):
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
    
        pos = list(pd.DataFrame(pos_tagger1.pos(t))[1])
        pos2 = list(pd.DataFrame(pos_tagger2.pos(t))[1])
        
        if len(pos) == 0 or len(pos2) == 0:
            print(str(idx)+"오류 제외")
        else :
            idx_list.append(idx)
    except:
        print(str(idx)+"오류 제외")
        
X_test = X_test[idx_list].tolist()



# Mecab 형태소 분석기 사용
t_result_list = []

# 형태소 분석결과 아무것도 안나오는 경우
for idx,t in enumerate(X_test):
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
        
        # 조사 제거
        pos = list(pd.DataFrame(pos_tagger1.pos(t))[1])
        t = np.array(pos_tagger.morphs(t))
        t = t.tolist()
        t_result_list.append(" ".join(t))
        
    except:
        print(t)
        print(str(idx)+" 제외")
        
X_test1 = t_result_list


t_result_list = []
for i,s in enumerate(X_test):
    s = str(s)
    s = s.strip()
    temp_list = s.split(" ")
    
   # 특수문자 제거  
    s = re.sub('#', '', s)
    s = re.sub('\'', '', s)
    s = re.sub('\「', '', s)
    s = re.sub('\」', '', s)
    s = re.sub('\,', '', s)
    s = re.sub('#', '', s)
    s = re.sub('@', '', s)
    s = re.sub('\+', '', s)
    s = re.sub('[A-z]', '', s)
    s = re.sub('[0-9]', '', s)
    s = s.strip()
    
    t_result_list.append(s)

X_all_list2 = t_result_list

for i, comment in enumerate(X_all_list2):
    temp = convert(comment)
    
    # 빈칸제거
    temp = temp.replace(" ","")
    temp2 = ""
    for t in temp:
        temp2 = temp2 + t +  " "
    temp2 = temp2[0:-1]
    X_all_list2[i] = temp2

X_test2 = X_all_list2

# Mecab 형태소 분석기 사용
t_result_list = []

# 형태소 분석결과 아무것도 안나오는 경우
for idx,t in enumerate(X_test):
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
        t = t.strip()
        
        # 조사 제거
        t = t.replace(" ", "") 
        t = list(t)
        t_result_list.append(" ".join(t))
        
    except:
        print(str(idx)+" 제외")

X_test3 = t_result_list

# Mecab 형태소 분석기 사용
t_result_list = []

# 형태소 분석결과 아무것도 안나오는 경우
for idx,t in enumerate(X_test):
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
        
        # 조사 제거
        pos = list(pd.DataFrame(pos_tagger2.pos(t))[1])
        t = np.array(pos_tagger2.morphs(t))
        t = t.tolist()
        t_result_list.append(" ".join(t))        
    except:
        print(t)
        #print(str(idx)+" 제외")

X_test4 = t_result_list

X_test1 = tk1.texts_to_sequences(X_test1)
X_test1 = sequence.pad_sequences(X_test1, maxlen=max_slen1,padding='pre')
X_test2 = tk2.texts_to_sequences(X_test2)
X_test2 = sequence.pad_sequences(X_test2, maxlen=max_slen2,padding='pre')
X_test3 = tk3.texts_to_sequences(X_test3)
X_test3 = sequence.pad_sequences(X_test3, maxlen=max_slen3,padding='pre')
X_test4 = tk4.texts_to_sequences(X_test4)
X_test4 = sequence.pad_sequences(X_test4, maxlen=max_slen4,padding='pre')

test_result = model.predict([X_test1,X_test2,X_test3,X_test4])
neg_prob = pd.DataFrame(test_result)[1]

test['prob'] = neg_prob

In [13]:
test = test.sort_values(['prob'],ascending=False)
test = test.reset_index(drop=True)
neg_test_result_df = test.loc[0:(n-1),['내용','prob']]
neg_test_result_df.to_excel('./sub_neg_df.xlsx')

/usr/local/lib/python3.6/dist-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.7.3' currently installed).
  warnings.warn(msg, UserWarning)


# 긍정 댓글 확률 계산

In [ ]:
test = pd.read_csv("./pos_df.csv")
X_test = test['내용']
X_test = X_test.drop_duplicates()
X_test = X_test.reset_index(drop=True)
idx_list = []

# 형태소 분석이 어려운 샘플은 제외
for idx,t in enumerate(X_test):
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
    
        pos = list(pd.DataFrame(pos_tagger1.pos(t))[1])
        pos2 = list(pd.DataFrame(pos_tagger2.pos(t))[1])
        
        if len(pos) == 0 or len(pos2) == 0:
            print(str(idx)+"오류 제외")
        else :
            idx_list.append(idx)
    except:
        print(str(idx)+"오류 제외")
        
X_test = X_test[idx_list].tolist()



# Mecab 형태소 분석기 사용
t_result_list = []

# 형태소 분석결과 아무것도 안나오는 경우
for idx,t in enumerate(X_test):
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
        
        # 조사 제거
        pos = list(pd.DataFrame(pos_tagger.pos(t))[1])
        t = np.array(pos_tagger.morphs(t))
        t = t.tolist()
        t_result_list.append(" ".join(t))
        
    except:
        print(t)
        print(str(idx)+" 제외")
        
X_test1 = t_result_list


t_result_list = []
for i,s in enumerate(X_test):
    s = str(s)
    s = s.strip()
    temp_list = s.split(" ")
    
   # 특수문자 제거  
    s = re.sub('#', '', s)
    s = re.sub('\'', '', s)
    s = re.sub('\「', '', s)
    s = re.sub('\」', '', s)
    s = re.sub('\,', '', s)
    s = re.sub('#', '', s)
    s = re.sub('@', '', s)
    s = re.sub('\+', '', s)
    s = re.sub('[A-z]', '', s)
    s = re.sub('[0-9]', '', s)
    s = s.strip()
    
    t_result_list.append(s)

X_all_list2 = t_result_list

for i, comment in enumerate(X_all_list2):
    temp = convert(comment)
    
    # 빈칸제거
    temp = temp.replace(" ","")
    temp2 = ""
    for t in temp:
        temp2 = temp2 + t +  " "
    temp2 = temp2[0:-1]
    X_all_list2[i] = temp2

X_test2 = X_all_list2

# Mecab 형태소 분석기 사용
t_result_list = []

# 형태소 분석결과 아무것도 안나오는 경우
for idx,t in enumerate(X_test):
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
        t = t.strip()
        
        # 조사 제거
        t = t.replace(" ", "") 
        t = list(t)
        t_result_list.append(" ".join(t))
        
    except:
        print(str(idx)+" 제외")

X_test3 = t_result_list

# Mecab 형태소 분석기 사용
t_result_list = []

# 형태소 분석결과 아무것도 안나오는 경우
for idx,t in enumerate(X_test):
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
        
        # 조사 제거
        pos = list(pd.DataFrame(pos_tagger2.pos(t))[1])
        t = np.array(pos_tagger2.morphs(t))
        t = t.tolist()
        t_result_list.append(" ".join(t))        
    except:
        print(t)
        #print(str(idx)+" 제외")

X_test4 = t_result_list

X_test1 = tk1.texts_to_sequences(X_test1)
X_test1 = sequence.pad_sequences(X_test1, maxlen=max_slen1,padding='pre')
X_test2 = tk2.texts_to_sequences(X_test2)
X_test2 = sequence.pad_sequences(X_test2, maxlen=max_slen2,padding='pre')
X_test3 = tk3.texts_to_sequences(X_test3)
X_test3 = sequence.pad_sequences(X_test3, maxlen=max_slen3,padding='pre')
X_test4 = tk4.texts_to_sequences(X_test4)
X_test4 = sequence.pad_sequences(X_test4, maxlen=max_slen4,padding='pre')

test_result = model.predict([X_test1,X_test2,X_test3,X_test4])
pos_prob = pd.DataFrame(test_result)[0]

test['prob'] = pos_prob

In [33]:
test

,Unnamed: 0,내용,키워드,prob
0,131292,"""아직도 금강산 개성공단으로 전력송출하는지 알고싶네요 억류자산도 우리자산인만큼 ...",기쁜|기쁨|기쁘다|이쁘다|예쁘다|어여쁘다|이쁨|어여쁨|예뿌다|이뿌다|신기|기대|존경...,0.851317
1,394932,그냥 컵짜파게티 사서 전자렌지에 이분 돌리고 쓰까주고 일분 돌리고 쓰까주고 일분 돌...,기쁜|기쁨|기쁘다|이쁘다|예쁘다|어여쁘다|이쁨|어여쁨|예뿌다|이뿌다|신기|기대|존경...,0.708149
2,146247,"""중부는 작년에 비하면 에어콘 하루도 안틀었는데 누군가 에어콘 회사랑 관련있나""",기쁜|기쁨|기쁘다|이쁘다|예쁘다|어여쁘다|이쁨|어여쁨|예뿌다|이뿌다|신기|기대|존경...,0.015528
3,154049,"아린 남동생 개잘생김 역시 유전자,,,",기쁜|기쁨|기쁘다|이쁘다|예쁘다|어여쁘다|이쁨|어여쁨|예뿌다|이뿌다|신기|기대|존경...,0.937515
4,398373,섹스를 하는 이유는 쾌락을 만족하기 위함과 자신의 유전자를 확실히 남기기 위함 두가...,기쁜|기쁨|기쁘다|이쁘다|예쁘다|어여쁘다|이쁨|어여쁨|예뿌다|이뿌다|신기|기대|존경...,0.962102
...,...,...,...,...
439089,287055,"""G7이 돈모아서 보내주면 되겠네 지들은 산업개발 다해놓곤 남은 하지 말라고?? 내...",기쁜|기쁨|기쁘다|이쁘다|예쁘다|어여쁘다|이쁨|어여쁨|예뿌다|이뿌다|신기|기대|존경...,0.997893
439090,32285,오! 나 서면자주가는데ㅋㅋ잘하면 만나겠다 ㅋㅋㅋㅋ사하구는 태풍때문에 피해많았어???,기쁜|기쁨|기쁘다|이쁘다|예쁘다|어여쁘다|이쁨|어여쁨|예뿌다|이뿌다|신기|기대|존경...,0.141282
439091,261099,방법 찾으면 제게도 알려주시길 청합니당 ㅎㅎ(플라스틱 병 말고 보온병 같은 건 어떨...,기쁜|기쁨|기쁘다|이쁘다|예쁘다|어여쁘다|이쁨|어여쁨|예뿌다|이뿌다|신기|기대|존경...,0.149044
439092,333285,@auddms0106 헐우리도글케바뀜근데 노랭이가아니고핫핑크ㅋㅋㅋ존나커다란플라스틱달...,기쁜|기쁨|기쁘다|이쁘다|예쁘다|어여쁘다|이쁨|어여쁨|예뿌다|이뿌다|신기|기대|존경...,0.972412


In [28]:
test = test.sort_values(['prob'],ascending=False)
test = test.reset_index(drop=True)
pos_test_result_df = test.loc[0:(n-1),['내용','prob']]
pos_test_result_df.to_excel('./sub_pos_df.xlsx')

/usr/local/lib/python3.6/dist-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.7.3' currently installed).
  warnings.warn(msg, UserWarning)
